In [43]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [44]:
def extract_frames_from_video(video_path, frame_count=10):
    """
    Extrai um número específico de frames de um vídeo.
    
    :param video_path: Caminho para o vídeo.
    :param frame_count: Número de frames a serem extraídos.
    :return: Lista de frames extraídos.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

In [45]:
def load_video_dataset(data_dir, frame_count=5):
    """
    Carrega o conjunto de dados de vídeos, extrai frames e atribui rótulos.
    
    :param data_dir: Diretório raiz contendo as subpastas dos vídeos.
    :param frame_count: Número de frames a serem extraídos de cada vídeo.
    :return: Arrays de dados e rótulos.
    """
    X, y = [], []
    class_labels = {'YouTube-real': 0, 'Celeb-synthesis': 1, 'Celeb-real': 0}
    
    for label, value in class_labels.items():
        subdir = os.path.join(data_dir, label)
        for file in os.listdir(subdir):
            if file.endswith('.mp4'):  # Supondo que os vídeos são arquivos .mp4
                video_path = os.path.join(subdir, file)
                frames = extract_frames_from_video(video_path, frame_count=frame_count)
                for frame in frames:
                    X.append(frame)
                    y.append(value)
    
    return X, y


In [46]:
def apply_sobel_filter(image, target_size=(256, 256)):
    """
    Aplica o filtro Sobel a uma imagem para detecção de bordas.

    :param image: Imagem a ser processada.
    :param target_size: Tamanho alvo para a imagem processada (largura, altura).
    :return: Imagem com filtro Sobel aplicado.
    """
    try:
        # Converter imagem para escala de cinza
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Aplicar filtros Sobel nas direções x e y
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        
        # Calcular magnitude da borda
        sobel = cv2.magnitude(sobelx, sobely)

        # Redimensionar a imagem para o tamanho alvo
        sobel_resized = cv2.resize(sobel, target_size)

        return sobel_resized
    
    except Exception as e:
        print(f"Erro ao aplicar o filtro Sobel: {e}")
        return None

In [47]:
def process_in_batches(X, batch_size=100, target_size=(256, 256)):
    """
    Processa dados em lotes para reduzir o consumo de memória.

    :param X: Conjunto de dados de imagens.
    :param batch_size: Tamanho do lote.
    :param target_size: Tamanho alvo para as imagens processadas (largura, altura).
    :return: Dados processados.
    """
    X_processed = []
    
    # Dividir dados em lotes e processar cada lote
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        
        # Aplicar filtro Sobel a cada imagem do lote
        batch_sobel = [apply_sobel_filter(frame, target_size) for frame in batch]
        
        # Filtrar imagens processadas corretamente
        batch_sobel = [sobel for sobel in batch_sobel if sobel is not None]
        
        X_processed.extend(batch_sobel)
        
    return np.array(X_processed)


In [48]:
def build_cnn(input_shape):
    """
    Constroi o modelo de CNN.
    
    :param input_shape: Forma da entrada do modelo.
    :return: Modelo CNN compilado.
    """
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [49]:
# Carregar o conjunto de dados de vídeos
data_dir = 'Celeb-DF'
X, y = load_video_dataset(data_dir)

In [50]:
# Processar em lotes
X_sobel = process_in_batches(X, batch_size=100, target_size=(256, 256))

# Normalizar os dados
X_sobel = X_sobel / np.max(X_sobel)

# Expandir as dimensões para incluir o canal de cor
X_sobel = np.expand_dims(X_sobel, axis=-1)

In [51]:
# Achatar as imagens de (256, 256, 1) para (256*256,)
n_samples = X_sobel.shape[0]
n_features = X_sobel.shape[1] * X_sobel.shape[2]
X_sobel_flattened = X_sobel.reshape(n_samples, n_features)

In [52]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sobel_flattened, y, test_size=0.2, random_state=42)

In [53]:
lda = LDA(n_components=1)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

In [54]:
# Expand dimensions for CNN input
X_train_lda = np.expand_dims(X_train_lda, axis=-1)
X_test_lda = np.expand_dims(X_test_lda, axis=-1)

In [55]:
def build_dense_model(input_shape):
    """
    Constroi um modelo denso para classificação após a aplicação de LDA.
    
    :param input_shape: Forma da entrada do modelo.
    :return: Modelo compilado.
    """
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [56]:
# Define input shape for dense model
input_shape = (X_train_lda.shape[1],)

# Construir e treinar o modelo denso
dense_model = build_dense_model(input_shape)
dense_model.fit(X_train_lda, np.array(y_train), epochs=10, batch_size=32, validation_data=(X_test_lda, np.array(y_test)))

c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9727 - loss: 0.1436 - val_accuracy: 0.7996 - val_loss: 1.6188
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9955 - loss: 0.0213 - val_accuracy: 0.7981 - val_loss: 1.5878
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9946 - loss: 0.0262 - val_accuracy: 0.8032 - val_loss: 1.5632
Epoch 4/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9958 - loss: 0.0254 - val_accuracy: 0.8068 - val_loss: 1.5329
Epoch 5/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.9979 - loss: 0.0168 - val_accuracy: 0.8176 - val_loss: 1.4615
Epoch 6/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.9965 - loss: 0.0208 - val_accuracy: 0.8176 - val_loss: 1.5099
Epoch 7/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9965 - loss: 0.0204 - val_accuracy: 0.8176 - val_loss: 1.4173
Epoch 8/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.9970 - loss: 0.0150 - val_accura

In [57]:
dense_model.save('Model/meu_modelo_sobel_LDA.h5')

In [58]:
def evaluate_model(model_path, X_test, y_test, class_names=['Class 0', 'Class 1']):
    # Load model
    model = load_model(model_path)
    
    # Predict
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    report = classification_report(y_test, y_pred, target_names=class_names)
    
    return accuracy, roc_auc, report

In [59]:
model_paths = ['Model/meu_modelo_sobel_LDA.h5']
for path in model_paths:
    accuracy, roc_auc, report = evaluate_model(path, X_test_lda, y_test)
    print(f"Modelo: {path}")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print("Relatório de Classificação:")
    print(report)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Modelo: Model/meu_modelo_sobel_LDA.h5
Acurácia: 0.8176
ROC-AUC: 0.7763
Relatório de Classificação:
              precision    recall  f1-score   support

     Class 0       0.83      0.59      0.69       475
     Class 1       0.81      0.94      0.87       912

    accuracy                           0.82      1387
   macro avg       0.82      0.76      0.78      1387
weighted avg       0.82      0.82      0.81      1387

